In [21]:
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.filters import median_filter
from skimage.morphology import disk
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import scipy.ndimage as nd
from PIL import Image
import pandas as pd
import numpy as np
import scipy
import os
import math

In [76]:
def getBaseColors(img, histRes, peaksDist):
    bins = np.arange(0, 256, 256/histRes)
    hist = np.histogram(img.ravel(), bins=bins, density=True)[0]

    # ищем локальные максимум на гистограмме
    peaks = scipy.signal.find_peaks(hist, distance=peaksDist)[0]
    
    step = 256 / res
    space = np.arange(step/2, 255-step/2, step).reshape((-1, 1))

    # идея в том, чтоб по количеству пиков на гистограмме определить количество основных цветов
    # а с помощью метода к-средних - найти сами цвета
    kmeans = KMeans(n_clusters=peaks.shape[0], random_state=0).fit(space, sample_weight=hist+0.000001)
    return np.int32(kmeans.cluster_centers_)

In [23]:
def approxImgWithColors(img, colors):
    approx = colors[0] * np.ones_like(img)
    diff = np.abs(img - approx)

    for col in colors[1:]:
        temp = np.abs(img - col)
        toChange = temp < diff
        diff[toChange] = temp[toChange]
        approx[toChange] = col
    
    return approx

In [24]:
def labelColorAreas(img, colors):
    labels = np.zeros_like(img)
    num = 1
    for col in colors:
        mask = img == col
        currLabels, n = nd.measurements.label(mask)
        currLabels += num
        labels[mask] = currLabels[mask]
        num += n+1
    return labels, num

In [25]:
def deleteArea(img, areaMask):
    tmpMask = nd.binary_dilation(areaMask, structure=np.ones((3, 3)))
    mask = np.logical_and(tmpMask, np.logical_not(areaMask))
    if np.sum(mask) == 0:
        return
    mainColor = img[mask]
    mainColor = np.bincount(mainColor)
    mainColor = np.argmax(mainColor)

    img[areaMask] = mainColor

In [26]:
def compositeImgs(background, upperLayer):
    background = background.convert("RGBA")
    upperLayer = upperLayer.convert("RGBA")
    upperLayer.putalpha(64)
    return Image.alpha_composite(background, upperLayer)

In [79]:
# directory with images
direct = "D:/Godis/Projects/SampleCoreAnalysisPresentation/"
res = 128
gaussFilterSgm = 0
medianFilterSize = 10
fragmentSize = 500 #px
colorsDist = 20 # 0..255
segmentMinSqr = 2000 #px

for name in os.listdir(direct+"photos"):
    orig = Image.open(direct+"photos/"+name)
    
    # можно уменьшить изображение, чтобы сегментация была быстрее
    # k - во сколько раз уменьшаем
    k = 3
    sizeLow = (int(orig.size[0]/k), int(orig.size[1]/k))
    origLow = orig.resize(sizeLow)
    
    img = np.copy(np.asarray(origLow)[:,:,0])
    
    approx = np.empty_like(img) # заводим буфер для картинки, аппроксимированный минимальным набором цветов
    approxNotFiltered = np.empty_like(img) 
    coloredSegments = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8) # буфер для раскрашенных сегментов

    # картинку разобьем на несколько прямоугольников и каждый независимо сегментируем
    fragmentsCnt = 1 + img.shape[0] // fragmentSize 
    for i in range(fragmentsCnt):
        a = fragmentSize*i
        b = min(fragmentSize*(i+1), img.shape[0])

        img[a:b] = median_filter(img[a:b], size=medianFilterSize)
        img[a:b] = gaussian_filter(img[a:b], sigma=gaussFilterSgm).astype(np.int32)
        
        # получаем набор цветов, которыми будем аппроксимровать картинку
        # идея в том, чтоб за основные цвета взять те, которые соответствуют пикам на гистограмме
        colors = getBaseColors(img[a:b], histRes=res, peaksDist=colorsDist)
        
        # аппроксимируем картинку небольшим количеством цветов
        approx[a:b] = approxImgWithColors(img[a:b], colors)

        # сегментируем картинку по связным областям одинакового цвета
        labels, num = labelColorAreas(approx[a:b], colors)

        approxNotFiltered[a:b] = approx[a:b]

        # удаляем сегменты с маленькой площадью
        for i in range(0, num):
            mask = labels == i+1
            square = np.count_nonzero(mask)
            if square == 0:
                continue
            elif square < segmentMinSqr:
                deleteArea(labels, mask)
                continue

        # раскрашиваем сегменты разными цветами
        for i in range(0, num):
            mask = labels == i+1
            if np.sum(mask) == 0:
                continue
            coloredSegments[a:b, :, 0][mask] = int(255 * np.random.rand())
            coloredSegments[a:b, :, 1][mask] = int(255 * np.random.rand())
            coloredSegments[a:b, :, 2][mask] = int(255 * np.random.rand())
    # можно переключить либо на вывод графиков
    # либо на вывод в файл
    segmentsImg = Image.fromarray(coloredSegments).convert("RGB")
    composed = compositeImgs(origLow, segmentsImg)
    if True:
        outImg = Image.new(size=(3*origLow.size[0], origLow.size[1]), mode="RGB")
        outImg.paste(origLow, (0, 0))
        outImg.paste(segmentsImg, (origLow.size[0], 0))
        outImg.paste(composed, (2*origLow.size[0], 0))
        outImg.save(direct+"segments/"+name, "PNG")
    else:
        fig, ax = plt.subplots(1, 3)
        ax[0].imshow(origLow)
        ax[1].imshow(segmentsImg)
        ax[2].imshow(composed)
